[![Ouvrir sur Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SkatAI/deeplearning/blob/master/notebooks/RNN_hands_on_claude_sunspots.ipynb)

# RNN et séries temporelles — Atelier pratique

**Deep Learning par la pratique — Alexis Perrier**

---

## Objectifs de cet atelier

1. **Comprendre les RNN** : comment un réseau de neurones traite des données séquentielles
2. **Prédire une série temporelle** : les taches solaires (sunspots)
3. **Comparer SimpleRNN, LSTM et GRU** : performance, vitesse, complexité
4. **Travailler avec l'IA** : utiliser Gemini pour générer, lire et comprendre du code

## Comment utiliser ce notebook

- Les cellules avec du code pré-écrit : **lisez, exécutez, observez**
- Les cellules marquées 🤖 **GEMINI** : demandez à Gemini de générer le code, puis lisez-le et demandez-lui de vous l'expliquer
- Les cellules marquées ❓ **QUESTION** : répondez en observant les résultats

**Utiliser l'IA pour coder n'est pas tricher — c'est la méthode de travail.**
Votre valeur n'est pas de taper du code, c'est de savoir quoi demander, comprendre ce qu'on vous donne, et juger si le résultat est bon.

---

## Étape 0 — Setup et exploration des données

On travaille avec le dataset **Sunspots** : le nombre mensuel de taches solaires observées depuis 1749.
Notre objectif : construire un modèle RNN qui prédit le nombre de taches solaires à partir des observations passées.

### 0.1 — Imports

In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense, Dropout

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import datetime, os

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

### 0.2 — Charger le dataset sunspots

Le dataset contient le nombre mensuel de taches solaires depuis 1749. C'est une série temporelle classique en science des données, avec une périodicité d'environ 11 ans (le cycle solaire).

In [ ]:
url = "https://raw.githubusercontent.com/SkatAI/deeplearning/master/data/sunspots.csv"

data = pd.read_csv(url)
data.columns = ['n', 'date', 'number']
data.index = data['date']

print(f"Nombre d'observations : {len(data)}")
print(f"Période : {data['date'].iloc[0]} → {data['date'].iloc[-1]}")
print()
data.head()

### 0.3 — Visualiser la série temporelle

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(data['number'], linewidth=0.8)
plt.title('Taches solaires — série complète')
plt.xlabel('Date')
plt.ylabel('Nombre de taches solaires')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 🤖 GEMINI — Explorer les données

Demandez à Gemini :

> *"Génère du code pour afficher un zoom sur 100 points consécutifs de la série `data['number']` (par exemple les indices 200 à 300) et calcule des statistiques de base : moyenne, écart-type, min, max. Utilise matplotlib. Qu'observes-tu sur la périodicité ?"*

Puis demandez :

> *"Explique-moi ce que tu observes. Y a-t-il un cycle visible ?"*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI


### ❓ QUESTION

En observant la série temporelle :
- La série est-elle régulière ? Y a-t-il une périodicité visible ?
- Les pics ont-ils tous la même amplitude ?
- Un humain pourrait-il prédire la suite de cette série ? Pourquoi ?

*(vos observations ici)*

---

## Étape 1 — Préparer les données pour un RNN

Un RNN ne prend pas un seul nombre en entrée : il prend une **séquence** de nombres. Pour prédire le nombre de taches solaires au mois N, on lui donne les valeurs des mois N-50 à N-1.

C'est le principe de la **fenêtre glissante** (sliding window).

### 1.1 — Split train/test (80/20)

In [ ]:
# 80% pour l'entraînement, 20% pour le test
training_data_len = math.ceil(len(data) * 0.8)

train_data = data[:training_data_len]['number']
test_data = data[training_data_len:]['number']

print(f"Train : {len(train_data)} observations")
print(f"Test  : {len(test_data)} observations")

### 1.2 — Normalisation avec MinMaxScaler

Les RNN fonctionnent mieux avec des valeurs entre 0 et 1. On utilise `MinMaxScaler` pour normaliser les données.

In [ ]:
# Reshape en 2D pour le scaler
dataset_train = train_data.values.reshape(-1, 1)
dataset_test = test_data.values.reshape(-1, 1)

# Normalisation
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(dataset_train)

# IMPORTANT : on utilise le même scaler pour le test
# (on ne fit pas un nouveau scaler sur le test !)
scaler_test = MinMaxScaler(feature_range=(0, 1))
scaled_test = scaler_test.fit_transform(dataset_test)

print(f"Train — min: {scaled_train.min():.2f}, max: {scaled_train.max():.2f}")
print(f"Test  — min: {scaled_test.min():.2f}, max: {scaled_test.max():.2f}")

### 1.3 — Créer les fenêtres glissantes

On crée des séquences de 50 pas de temps. Pour chaque fenêtre de 50 valeurs, la cible est la valeur suivante (la 51ème).

```
Fenêtre 1 : [v0, v1, ..., v49]  →  cible : v50
Fenêtre 2 : [v1, v2, ..., v50]  →  cible : v51
Fenêtre 3 : [v2, v3, ..., v51]  →  cible : v52
...
```

In [ ]:
WINDOW_SIZE = 50

def create_sequences(data, window_size):
    """Crée les fenêtres glissantes X et les cibles y."""
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data[i - window_size:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(scaled_train, WINDOW_SIZE)
X_test, y_test = create_sequences(scaled_test, WINDOW_SIZE)

print(f"X_train : {X_train.shape}, y_train : {y_train.shape}")
print(f"X_test  : {X_test.shape}, y_test  : {y_test.shape}")

### 1.4 — Reshape en tenseur 3D

Un RNN attend des données au format **(n_samples, timesteps, features)** :
- `n_samples` : nombre de fenêtres
- `timesteps` : taille de la fenêtre (50)
- `features` : nombre de variables par pas de temps (1, car série univariée)

In [ ]:
# Reshape pour le RNN : (samples, timesteps, features)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
y_train = y_train.reshape(y_train.shape[0], 1)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_test = y_test.reshape(y_test.shape[0], 1)

print(f"X_train : {X_train.shape}  →  (samples, timesteps, features)")
print(f"y_train : {y_train.shape}")
print(f"X_test  : {X_test.shape}")
print(f"y_test  : {y_test.shape}")

### 🤖 GEMINI — Comprendre le format des données

Demandez à Gemini :

> *"Explique-moi pourquoi un RNN a besoin de données au format (n_samples, timesteps, features). Que représente chaque dimension ? Pourquoi utilise-t-on une fenêtre glissante pour transformer une série temporelle en données d'entraînement ?"*

### ❓ QUESTION

- Combien d'échantillons d'entraînement avons-nous ? Combien de test ?
- Que se passerait-il si on augmentait la taille de la fenêtre à 100 ? Et si on la diminuait à 10 ?
- Pourquoi perd-on les 50 premières observations ?

*(vos réponses ici)*

---

## Fonctions utilitaires

On définit deux fonctions qu'on réutilisera pour chaque modèle.

In [ ]:
def plot_training(history, title):
    """Courbes de loss train/validation."""
    plt.figure(figsize=(10, 4))
    plt.plot(history.history['loss'], label='Train', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation', linewidth=2)
    plt.title(f'{title} — Loss (MSE)')
    plt.xlabel('Époque')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


def plot_predictions(y_true, y_pred, title):
    """Plot prédictions vs réalité sur le test set."""
    plt.figure(figsize=(14, 5))
    plt.plot(y_true, label='Réalité', linewidth=1.5)
    plt.plot(y_pred, label='Prédictions', linewidth=1.5, alpha=0.8)
    plt.title(f'{title} — Prédictions vs Réalité')
    plt.xlabel('Observations (test set)')
    plt.ylabel('Nombre de taches solaires (normalisé)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

print("Fonctions plot_training() et plot_predictions() définies.")

---

## Étape 2 — Baseline : SimpleRNN

On commence par le modèle le plus simple : un **SimpleRNN**. C'est un RNN classique où chaque neurone reçoit l'entrée courante et l'état caché précédent.

### 2.1 — Construire le modèle SimpleRNN

In [ ]:
import time

simple_rnn_model = Sequential([
    SimpleRNN(50, activation='relu', input_shape=(WINDOW_SIZE, 1)),
    Dense(1)
])

simple_rnn_model.compile(optimizer='adam', loss='mse')

simple_rnn_model.summary()

### 🤖 GEMINI — Comprendre l'architecture

Demandez à Gemini :

> *"Explique-moi le model.summary() du SimpleRNN ci-dessus. Combien de paramètres a-t-il ? Comment un SimpleRNN traite-t-il une séquence pas à pas ? Que signifie le chiffre 50 dans SimpleRNN(50) ?"*

### 2.2 — Entraîner le SimpleRNN

In [ ]:
print("Entraînement du SimpleRNN...")

start_time = time.time()

history_simple = simple_rnn_model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1
)

time_simple = time.time() - start_time
print(f"\nTemps d'entraînement : {time_simple:.1f} secondes")

### 2.3 — Visualiser les résultats

In [ ]:
# Courbes de loss
plot_training(history_simple, "SimpleRNN")

# Prédictions sur le test set
y_pred_simple = simple_rnn_model.predict(X_test)

plot_predictions(y_test, y_pred_simple, "SimpleRNN")

# RMSE
rmse_simple = np.sqrt(mean_squared_error(y_test.flatten(), y_pred_simple.flatten()))
print(f"RMSE SimpleRNN : {rmse_simple:.4f}")

### ❓ QUESTION

Observez les courbes et les prédictions :
- Le modèle capture-t-il la tendance générale de la série ?
- Les pics (maxima) sont-ils bien prédits ?
- Y a-t-il de l'overfitting ? (la loss de validation remonte-t-elle ?)
- Quelle est la RMSE ?

*(vos observations ici)*

---

## Étape 3 — LSTM

### 3.1 — Pourquoi LSTM ?

Le **SimpleRNN** souffre du problème du **vanishing gradient** : quand les séquences sont longues, le gradient devient très petit lors de la rétropropagation, et le modèle "oublie" les informations anciennes.

Le **LSTM** (Long Short-Term Memory) résout ce problème grâce à des **portes** (gates) qui contrôlent le flux d'information :
- **Porte d'oubli** (forget gate) : quelles informations effacer de la mémoire
- **Porte d'entrée** (input gate) : quelles nouvelles informations stocker
- **Porte de sortie** (output gate) : quelles informations transmettre

Résultat : le LSTM peut mémoriser des dépendances sur de longues séquences.

### 🤖 GEMINI — Construire le modèle LSTM

Demandez à Gemini :

> *"Réécris le modèle SimpleRNN de l'étape 2 en remplaçant SimpleRNN par LSTM. Garde la même structure : 1 couche LSTM(50), Dense(1), compile avec Adam et MSE. Entraîne pendant 20 epochs avec batch_size=32 et validation_data=(X_test, y_test). Mesure le temps d'entraînement avec time.time(). Stocke le modèle dans `lstm_model`, l'historique dans `history_lstm` et le temps dans `time_lstm`."*

Puis demandez :

> *"Combien de paramètres a le LSTM par rapport au SimpleRNN ? Pourquoi cette différence ?"*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI


### 3.2 — Visualiser les résultats du LSTM

In [ ]:
# Courbes de loss
plot_training(history_lstm, "LSTM")

# Prédictions sur le test set
y_pred_lstm = lstm_model.predict(X_test)

plot_predictions(y_test, y_pred_lstm, "LSTM")

# RMSE
rmse_lstm = np.sqrt(mean_squared_error(y_test.flatten(), y_pred_lstm.flatten()))
print(f"RMSE LSTM : {rmse_lstm:.4f}")

### ❓ QUESTION

Comparez le LSTM avec le SimpleRNN :
- Le LSTM fait-il mieux que le SimpleRNN ? Le RMSE a-t-il diminué ?
- L'entraînement est-il plus lent ? De combien ?
- Les prédictions sur les pics sont-elles meilleures ?

*(vos observations ici)*

---

## Étape 4 — Comparer les deux approches

On met côte à côte les résultats du SimpleRNN et du LSTM.

In [ ]:
# Comparaison visuelle
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# SimpleRNN
axes[0].plot(y_test, label='Réalité', linewidth=1.5)
axes[0].plot(y_pred_simple, label='SimpleRNN', linewidth=1.5, alpha=0.8)
axes[0].set_title(f'SimpleRNN — RMSE: {rmse_simple:.4f}')
axes[0].set_xlabel('Observations')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# LSTM
axes[1].plot(y_test, label='Réalité', linewidth=1.5)
axes[1].plot(y_pred_lstm, label='LSTM', linewidth=1.5, alpha=0.8)
axes[1].set_title(f'LSTM — RMSE: {rmse_lstm:.4f}')
axes[1].set_xlabel('Observations')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Tableau récapitulatif
print("=" * 60)
print("COMPARAISON SimpleRNN vs LSTM")
print("=" * 60)
print(f"{'Métrique':<30} {'SimpleRNN':>12} {'LSTM':>12}")
print("-" * 60)
print(f"{'RMSE':<30} {rmse_simple:>12.4f} {rmse_lstm:>12.4f}")
print(f"{'Paramètres':<30} {simple_rnn_model.count_params():>12,} {lstm_model.count_params():>12,}")
print(f"{'Temps entraînement (s)':<30} {time_simple:>12.1f} {time_lstm:>12.1f}")
print("=" * 60)

### ❓ QUESTIONS

Observez le tableau et les graphiques :

1. **Lequel est meilleur** en termes de RMSE ?
2. **Le gain de performance justifie-t-il** la complexité supplémentaire (plus de paramètres, entraînement plus long) ?
3. **À partir de quelle epoch** les courbes de validation stagnent-elles pour chaque modèle ?

*(vos réponses ici)*

---

## Étape 5 — Expérimenter

Maintenant c'est à vous de jouer. Choisissez **une ou plusieurs** expérimentations parmi les suivantes.

Pour chaque expérimentation :
1. Demandez à Gemini de générer le code
2. Lisez le code et demandez à Gemini de vous l'expliquer
3. Exécutez et observez les résultats
4. Notez vos conclusions

---

### Expérimentation A — GRU

Le **GRU** (Gated Recurrent Unit) est un compromis entre SimpleRNN et LSTM : il a des portes comme le LSTM, mais avec une architecture plus simple (2 portes au lieu de 3). Il est souvent aussi performant que le LSTM mais plus rapide à entraîner.

### 🤖 GEMINI

Demandez à Gemini :

> *"Crée un modèle GRU avec la même structure que le LSTM : 1 couche GRU(50), Dense(1), compile avec Adam et MSE. Entraîne pendant 20 epochs avec batch_size=32 et validation_data=(X_test, y_test). Mesure le temps d'entraînement. Stocke dans `gru_model`, `history_gru`, `time_gru`. Affiche le summary."*

Puis demandez :

> *"Quelle est la différence entre GRU et LSTM ? Lequel a le plus de paramètres ? Pourquoi ?"*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI


In [ ]:
# Visualiser les résultats du GRU
plot_training(history_gru, "GRU")

y_pred_gru = gru_model.predict(X_test)
plot_predictions(y_test, y_pred_gru, "GRU")

rmse_gru = np.sqrt(mean_squared_error(y_test.flatten(), y_pred_gru.flatten()))
print(f"RMSE GRU : {rmse_gru:.4f}")
print(f"Temps d'entraînement : {time_gru:.1f}s")

### ❓ QUESTION

- Le GRU est-il plus rapide que le LSTM ? De combien ?
- La performance (RMSE) est-elle comparable ?
- Combien de paramètres a le GRU par rapport au LSTM ?

*(vos observations ici)*

---

### Expérimentation B — Empiler des couches

### 🤖 GEMINI

Demandez à Gemini :

> *"Crée un modèle avec 2 couches LSTM empilées : LSTM(50, return_sequences=True) pour la première couche, LSTM(50) pour la deuxième, puis Dense(1). Attention : la première couche doit avoir `return_sequences=True` pour transmettre la séquence complète à la couche suivante. Compile avec Adam et MSE. Entraîne 20 epochs, batch_size=32, validation_data=(X_test, y_test). Stocke dans `stacked_model`, `history_stacked`, `time_stacked`. Affiche le summary."*

Puis demandez :

> *"Pourquoi faut-il `return_sequences=True` sur la première couche LSTM mais pas sur la dernière ?"*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI


In [ ]:
# Visualiser les résultats du modèle empilé
plot_training(history_stacked, "LSTM empilé (2 couches)")

y_pred_stacked = stacked_model.predict(X_test)
plot_predictions(y_test, y_pred_stacked, "LSTM empilé (2 couches)")

rmse_stacked = np.sqrt(mean_squared_error(y_test.flatten(), y_pred_stacked.flatten()))
print(f"RMSE LSTM empilé : {rmse_stacked:.4f}")
print(f"Paramètres : {stacked_model.count_params():,}")

### ❓ QUESTION

- Le modèle empilé est-il meilleur que le LSTM à 1 couche ?
- Combien de paramètres supplémentaires a-t-il ?
- L'ajout de profondeur aide-t-il toujours ?

*(vos observations ici)*

---

### Expérimentation C — Ajouter du Dropout

### 🤖 GEMINI

Demandez à Gemini :

> *"Crée un modèle LSTM avec Dropout : LSTM(50), Dropout(0.2), Dense(1). Compile avec Adam et MSE. Entraîne 20 epochs, batch_size=32, validation_data=(X_test, y_test). Stocke dans `dropout_model`, `history_dropout`, `time_dropout`. Affiche le summary."*

Puis demandez :

> *"Que fait le Dropout ? Pourquoi aide-t-il à réduire l'overfitting ? Que signifie Dropout(0.2) ?"*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI


In [ ]:
# Visualiser les résultats du modèle avec Dropout
plot_training(history_dropout, "LSTM + Dropout")

y_pred_dropout = dropout_model.predict(X_test)
plot_predictions(y_test, y_pred_dropout, "LSTM + Dropout")

rmse_dropout = np.sqrt(mean_squared_error(y_test.flatten(), y_pred_dropout.flatten()))
print(f"RMSE LSTM + Dropout : {rmse_dropout:.4f}")

### ❓ QUESTION

- L'écart entre loss train et loss validation a-t-il diminué avec le Dropout ?
- La RMSE est-elle meilleure ou moins bonne ?
- Le Dropout régularise-t-il efficacement ce modèle ?

*(vos observations ici)*

---

### Expérimentation D — Changer la taille de la fenêtre

### 🤖 GEMINI

Demandez à Gemini :

> *"Teste le modèle LSTM avec deux tailles de fenêtre différentes : window=20 et window=100. Pour chaque taille :
> 1. Recrée les séquences X_train et X_test avec `create_sequences` et la nouvelle taille de fenêtre
> 2. Reshape les données en tenseur 3D
> 3. Crée un LSTM(50) + Dense(1), compile avec Adam et MSE
> 4. Entraîne 20 epochs, batch_size=32
> 5. Calcule la RMSE
>
> Affiche les prédictions des deux modèles côte à côte avec les RMSE dans les titres. Utilise `scaled_train` et `scaled_test` comme données source."*

Puis demandez :

> *"Comment la taille de la fenêtre affecte-t-elle les prédictions ? Pourquoi une fenêtre trop petite est problématique ? Et trop grande ?"*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI


### ❓ QUESTION

- Quelle taille de fenêtre donne les meilleurs résultats ?
- Sachant que le cycle solaire dure environ 11 ans (~132 mois), une fenêtre de 50 capture-t-elle un cycle complet ?
- Y a-t-il un trade-off entre la taille de la fenêtre et le nombre d'échantillons d'entraînement ?

*(vos observations ici)*

---

## Étape 6 — TensorBoard

TensorBoard permet de visualiser l'entraînement en temps réel : loss, poids, distributions. On va ré-entraîner notre meilleur modèle avec un callback TensorBoard.

In [ ]:
%load_ext tensorboard

In [ ]:
# Créer un nouveau modèle LSTM pour TensorBoard
tb_model = Sequential([
    LSTM(50, activation='relu', input_shape=(WINDOW_SIZE, 1)),
    Dense(1)
])

tb_model.compile(optimizer='adam', loss='mse')

# Callback TensorBoard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

print(f"Logs TensorBoard dans : {logdir}")
print("Entraînement avec TensorBoard...")

tb_model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[tensorboard_callback]
)

In [ ]:
%tensorboard --logdir logs

### 🤖 GEMINI — Comprendre TensorBoard

Demandez à Gemini :

> *"Explique-moi ce que montrent les onglets Scalars et Histograms dans TensorBoard. À quoi servent-ils pour le diagnostic d'un modèle de deep learning ?"*

---

## Étape 7 — Synthèse

Vous avez construit et comparé plusieurs approches pour prédire les taches solaires :

| | SimpleRNN | LSTM | GRU |
|---|---|---|---|
| **Principe** | RNN basique, pas de portes | 3 portes (oubli, entrée, sortie) | 2 portes (reset, update) |
| **Mémoire longue** | Faible (vanishing gradient) | Bonne | Bonne |
| **Paramètres** | Le moins | Le plus (~4x SimpleRNN) | Intermédiaire (~3x SimpleRNN) |
| **Vitesse** | Le plus rapide | Le plus lent | Intermédiaire |
| **Cas d'usage** | Séquences courtes | Séquences longues, dépendances complexes | Bon compromis performance/vitesse |

### ❓ QUESTIONS FINALES

1. **Cas d'usage** : dans votre domaine professionnel, imaginez un cas d'usage de RNN sur une série temporelle. Décrivez-le en 2-3 phrases.

2. **Conseil** : un collègue veut prédire des ventes mensuelles avec seulement 100 points de données. Que lui conseillez-vous ? Quel modèle ? Quelle taille de fenêtre ?

3. **Limites** : pourquoi le modèle a-t-il du mal avec les pics extrêmes de la série ? Que pourrait-on faire pour améliorer les prédictions ?

4. **Surprise** : qu'est-ce qui vous a le plus surpris dans cet atelier ?

*(vos réponses ici)*

---

## Pour aller plus loin (optionnel)

Si vous avez du temps et de la curiosité, voici quelques pistes d'exploration. Utilisez Gemini pour vous guider.

- **Prédiction multi-step** : au lieu de prédire 1 seul point, prédire les 10 prochains points. Demandez à Gemini *"Modifie le modèle pour qu'il prédise les 10 prochains points au lieu d'un seul. Comment faut-il changer la couche Dense de sortie et les cibles y ?"*
- **Bidirectional LSTM** : demandez à Gemini *"Enveloppe la couche LSTM dans un `Bidirectional()`. Quel avantage cela apporte-t-il pour une série temporelle ?"*
- **Autre dataset** : essayez avec des données de température, de trafic web ou de cours de bourse
- **Attention mechanism** : demandez à Gemini *"Ajoute une couche d'attention après le LSTM. Qu'est-ce que cela change ?"*

In [ ]:
# VOTRE EXPLORATION ICI
